In [ ]:
# ============================================================
# 1. SETUP Y CONFIGURACIÓN
# ============================================================
import sys
import os
import json
import time
import statistics
from pathlib import Path
from datetime import datetime
from collections import Counter

# Detectar entorno
IN_COLAB = 'google.colab' in sys.modules
print(f"🖥️  Entorno: {'Google Colab' if IN_COLAB else 'Local (VS Code)'}")

# Resolver paths
if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        BASE_DIR = Path('/content/drive/MyDrive/OKLA/chatbot-llm/FASE_5_MEJORA_CONTINUA')
    except ImportError:
        BASE_DIR = Path('.').resolve()
else:
    BASE_DIR = Path('.').resolve()
    if not (BASE_DIR / 'evaluation').exists():
        for candidate in [BASE_DIR, BASE_DIR.parent / 'FASE_5_MEJORA_CONTINUA']:
            if (candidate / 'evaluation').exists():
                BASE_DIR = candidate
                break

# Agregar paths para imports
for subdir in ['evaluation', 'feedback', 'retrain', 'monitoring', 'ab_testing']:
    path = str(BASE_DIR / subdir)
    if path not in sys.path:
        sys.path.insert(0, path)
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

# ── Configuración del LLM Server ──────────────────────────
LLM_SERVER_URL = "http://localhost:8000"  # Cambiar si es diferente
TEST_DATA_PATH = str(BASE_DIR.parent / 'FASE_2_DATASET' / 'output' / 'okla_test.jsonl')
RESULTS_DIR = BASE_DIR / 'results'
RESULTS_DIR.mkdir(exist_ok=True)

print(f"📂 Base:       {BASE_DIR}")
print(f"🌐 LLM Server: {LLM_SERVER_URL}")
print(f"📊 Test data:  {TEST_DATA_PATH}")
print(f"📁 Results:    {RESULTS_DIR}")

In [ ]:
# ============================================================
# 1b. VERIFICAR CONEXIÓN AL LLM SERVER
# ============================================================
import urllib.request
import urllib.error

def check_llm_server(url: str) -> dict:
    """Verifica que el LLM server está corriendo."""
    try:
        req = urllib.request.Request(f"{url}/health")
        with urllib.request.urlopen(req, timeout=5) as resp:
            data = json.loads(resp.read())
            return {'status': 'ok', 'data': data}
    except urllib.error.URLError as e:
        return {'status': 'error', 'error': str(e)}
    except Exception as e:
        return {'status': 'error', 'error': str(e)}

result = check_llm_server(LLM_SERVER_URL)

if result['status'] == 'ok':
    d = result['data']
    print(f"✅ LLM Server conectado")
    print(f"   Status:    {d.get('status', 'N/A')}")
    print(f"   Model:     {d.get('model_loaded', 'N/A')}")
    print(f"   Uptime:    {d.get('uptime_seconds', 0):.0f}s")
    print(f"   Requests:  {d.get('total_requests', 0):,}")
    SERVER_AVAILABLE = True
else:
    print(f"⚠️  LLM Server no disponible en {LLM_SERVER_URL}")
    print(f"   Error: {result['error']}")
    print(f"\n   Las secciones que requieren el server se saltarán.")
    print(f"   Para iniciar el server:")
    print(f"   cd backend/ChatbotService/LlmServer && python server.py")
    SERVER_AVAILABLE = False

---
## 2️⃣ Evaluación del Modelo

Evalúa el modelo contra el test set en 5 dimensiones:
intent accuracy, latencia, seguridad, naturalidad RD, lead capture.

> ⚠️ **Requiere:** LLM server corriendo

In [ ]:
# ============================================================
# 2. EVALUACIÓN DEL MODELO
# ============================================================
if not SERVER_AVAILABLE:
    print("⏭️  Saltando evaluación — LLM server no disponible")
else:
    from evaluate_model import (
        evaluate_intent_accuracy,
        evaluate_latency,
        evaluate_safety,
        evaluate_rd_naturalness,
        evaluate_lead_capture,
    )
    
    # Cargar test data
    test_samples = []
    with open(TEST_DATA_PATH) as f:
        for line in f:
            test_samples.append(json.loads(line))
    print(f"📊 Test set cargado: {len(test_samples)} conversaciones")
    
    # Limitar a N muestras para rapidez
    MAX_EVAL_SAMPLES = 50  # Ajustar según tiempo disponible
    eval_samples = test_samples[:MAX_EVAL_SAMPLES]
    print(f"   Evaluando {len(eval_samples)} muestras...\n")
    
    # ── Intent Accuracy ────────────────────────────────────
    print("🎯 Evaluando Intent Accuracy...")
    intent_results = evaluate_intent_accuracy(eval_samples, LLM_SERVER_URL)
    print(f"   Accuracy: {intent_results['accuracy']:.1%}")
    print(f"   Pass: {'✅' if intent_results['accuracy'] >= 0.70 else '❌'}")
    
    # ── Latencia ───────────────────────────────────────────
    print("\n⚡ Evaluando Latencia...")
    latency_results = evaluate_latency(eval_samples[:20], LLM_SERVER_URL)
    print(f"   p50: {latency_results['p50']:.0f}ms")
    print(f"   p90: {latency_results['p90']:.0f}ms")
    print(f"   p95: {latency_results['p95']:.0f}ms")
    print(f"   Pass: {'✅' if latency_results['p95'] < 10000 else '❌'}")
    
    # ── Safety ─────────────────────────────────────────────
    print("\n🛡️ Evaluando Safety...")
    safety_results = evaluate_safety(LLM_SERVER_URL)
    print(f"   Violations: {safety_results['violations']}")
    print(f"   Pass: {'✅' if safety_results['violations'] == 0 else '❌'}")
    
    # ── Resumen ────────────────────────────────────────────
    print(f"\n{'='*50}")
    print(f"📊 RESUMEN DE EVALUACIÓN")
    print(f"{'='*50}")
    all_pass = (
        intent_results['accuracy'] >= 0.70 and
        latency_results['p95'] < 10000 and
        safety_results['violations'] == 0
    )
    print(f"   Veredicto: {'✅ PASS' if all_pass else '❌ NEEDS IMPROVEMENT'}")

---
## 3️⃣ Análisis de Feedback

Analiza el feedback recolectado de usuarios para identificar áreas de mejora.

In [ ]:
# ============================================================
# 3. ANÁLISIS DE FEEDBACK
# ============================================================
from feedback_system import FeedbackCollector, FeedbackAnalyzer

FEEDBACK_DIR = BASE_DIR / 'feedback' / 'feedback_data'

if FEEDBACK_DIR.exists() and any(FEEDBACK_DIR.glob('*.jsonl')):
    collector = FeedbackCollector(str(FEEDBACK_DIR))
    all_feedback = collector.load_all()
    
    print(f"💬 Feedback recolectado: {len(all_feedback)} entradas")
    
    if len(all_feedback) > 0:
        # Stats básicos
        stats = collector.get_stats()
        print(f"\n📊 Estadísticas:")
        print(f"   Promedio rating: {stats.get('avg_rating', 'N/A')}")
        print(f"   👍 Positivos: {stats.get('thumbs_up', 0)}")
        print(f"   👎 Negativos: {stats.get('thumbs_down', 0)}")
        
        # Análisis profundo
        analyzer = FeedbackAnalyzer(all_feedback)
        
        weak_intents = analyzer.identify_weak_intents()
        if weak_intents:
            print(f"\n⚠️ Intents débiles:")
            for intent, score in weak_intents[:5]:
                print(f"   • {intent}: {score:.2f}")
        
        priorities = analyzer.generate_improvement_priorities()
        if priorities:
            print(f"\n🎯 Prioridades de mejora:")
            for i, p in enumerate(priorities[:5], 1):
                print(f"   {i}. {p}")
else:
    print(f"📭 No hay feedback recolectado aún.")
    print(f"   Directorio esperado: {FEEDBACK_DIR}")
    print(f"\n   El feedback se acumula cuando los usuarios interactúan")
    print(f"   con el chatbot y califican las respuestas.")

---
## 4️⃣ Detección de Drift

Compara métricas actuales vs baseline para detectar degradación.

> ⚠️ **Requiere:** LLM server corriendo + baseline establecido

In [ ]:
# ============================================================
# 4. DETECCIÓN DE DRIFT
# ============================================================
from drift_detector import establish_baseline, detect_drift

BASELINE_PATH = RESULTS_DIR / 'baseline.json'

if not SERVER_AVAILABLE:
    print("⏭️  Saltando drift detection — LLM server no disponible")
elif not BASELINE_PATH.exists():
    print("📏 No hay baseline establecido. Creando baseline inicial...")
    print(f"   Usando {min(30, len(test_samples))} muestras del test set...")
    
    baseline = establish_baseline(
        server_url=LLM_SERVER_URL,
        test_samples=test_samples[:30],
    )
    
    with open(BASELINE_PATH, 'w') as f:
        json.dump(baseline, f, indent=2)
    
    print(f"\n✅ Baseline guardado en: {BASELINE_PATH}")
    print(f"   Confidence avg: {baseline.get('avg_confidence', 'N/A'):.3f}")
    print(f"   Latency p95:    {baseline.get('latency_p95', 'N/A'):.0f}ms")
    print(f"   Fallback rate:  {baseline.get('fallback_rate', 'N/A'):.1%}")
else:
    print("📏 Baseline encontrado. Comparando con métricas actuales...")
    
    with open(BASELINE_PATH) as f:
        baseline = json.load(f)
    
    drift_results = detect_drift(
        server_url=LLM_SERVER_URL,
        test_samples=test_samples[:30],
        baseline=baseline,
    )
    
    if drift_results.get('drift_detected', False):
        print(f"\n🚨 DRIFT DETECTADO")
        for signal in drift_results.get('signals', []):
            print(f"   ⚠️ {signal['type']}: {signal['description']}")
        print(f"\n   Recomendación: Ejecutar pipeline de re-entrenamiento")
    else:
        print(f"\n✅ No se detectó drift — Modelo estable")

---
## 5️⃣ A/B Testing

Gestiona experimentos A/B entre versiones del modelo.

In [ ]:
# ============================================================
# 5. A/B TESTING
# ============================================================
from ab_testing import ABExperiment

EXPERIMENTS_DIR = RESULTS_DIR / 'experiments'
EXPERIMENTS_DIR.mkdir(exist_ok=True)

# Buscar experimentos activos
active_experiments = list(EXPERIMENTS_DIR.glob('*.json'))

if active_experiments:
    print(f"🔬 Experimentos encontrados: {len(active_experiments)}")
    for exp_path in active_experiments:
        with open(exp_path) as f:
            exp_data = json.load(f)
        
        name = exp_data.get('name', 'unknown')
        control = exp_data.get('control_model', 'N/A')
        treatment = exp_data.get('treatment_model', 'N/A')
        n_results = len(exp_data.get('results', []))
        min_samples = exp_data.get('min_samples', 100)
        
        print(f"\n   📋 {name}")
        print(f"      Control:   {control}")
        print(f"      Treatment: {treatment}")
        print(f"      Samples:   {n_results}/{min_samples}")
        
        if n_results >= min_samples:
            print(f"      Status:    ✅ Listo para analizar")
        else:
            print(f"      Status:    ⏳ Recolectando ({100*n_results/min_samples:.0f}%)")
else:
    print(f"📭 No hay experimentos A/B activos.")
    print(f"\n   Para crear uno:")
    print(f"   python ab_testing/ab_testing.py create \\")
    print(f"     --name 'v1-vs-v2' \\")
    print(f"     --control 'okla-v1.0' \\")
    print(f"     --treatment 'okla-v2.0' \\")
    print(f"     --traffic-split 50")

---
## 6️⃣ Decisión de Re-entrenamiento

Evalúa si es necesario re-entrenar el modelo basado en feedback, drift y edad.

In [ ]:
# ============================================================
# 6. DECISIÓN DE RE-ENTRENAMIENTO
# ============================================================
from retrain_pipeline import ModelVersionManager

VERSIONS_PATH = RESULTS_DIR / 'model_versions.json'

manager = ModelVersionManager(str(VERSIONS_PATH))

print("🔄 Estado del Pipeline de Re-entrenamiento")
print("=" * 50)

# Verificar criterios
should_retrain, reasons = manager.should_retrain(
    feedback_dir=str(FEEDBACK_DIR) if FEEDBACK_DIR.exists() else None,
)

if should_retrain:
    print(f"\n🚨 RE-ENTRENAMIENTO RECOMENDADO")
    for reason in reasons:
        print(f"   • {reason}")
    print(f"\n   Próximos pasos:")
    print(f"   1. Ejecutar: python retrain/retrain_pipeline.py collect")
    print(f"   2. Ejecutar: python retrain/retrain_pipeline.py prepare")
    print(f"   3. Abrir FASE_3 notebook → Select Kernel → Colab → GPU")
    print(f"   4. Entrenar con datos actualizados")
else:
    print(f"\n✅ No se requiere re-entrenamiento por ahora")
    if reasons:
        for reason in reasons:
            print(f"   ℹ️ {reason}")

# Mostrar versiones registradas
versions = manager.list_versions()
if versions:
    print(f"\n📦 Versiones de modelo registradas:")
    for v in versions:
        status_icon = {'candidate': '🟡', 'promoted': '🟢', 'retired': '⚪'}
        icon = status_icon.get(v.get('status', ''), '❓')
        print(f"   {icon} {v['version']} — {v['status']} ({v.get('date', 'N/A')})")

---
## 7️⃣ Monitoreo Prometheus

Estado de las métricas Prometheus del LLM server.

In [ ]:
# ============================================================
# 7. VERIFICAR MÉTRICAS PROMETHEUS
# ============================================================

if not SERVER_AVAILABLE:
    print("⏭️  Saltando Prometheus — LLM server no disponible")
else:
    try:
        req = urllib.request.Request(f"{LLM_SERVER_URL}/metrics")
        with urllib.request.urlopen(req, timeout=5) as resp:
            metrics_text = resp.read().decode('utf-8')
        
        print("📡 Métricas Prometheus disponibles en /metrics")
        print(f"   URL: {LLM_SERVER_URL}/metrics")
        print()
        
        # Parsear métricas clave
        key_metrics = [
            'okla_llm_requests_total',
            'okla_llm_requests_success_total',
            'okla_llm_avg_response_time_ms',
            'okla_llm_uptime_seconds',
            'okla_llm_model_loaded',
        ]
        
        print("   Métricas clave:")
        for line in metrics_text.split('\n'):
            for metric in key_metrics:
                if line.startswith(metric) and not line.startswith('#'):
                    parts = line.split()
                    if len(parts) >= 2:
                        name = parts[0].split('{')[0]
                        value = parts[-1]
                        print(f"   • {name}: {value}")
        
        print(f"\n   📊 Dashboard Grafana:")
        print(f"   Importar: monitoring/grafana-dashboard.json")
        
    except Exception as e:
        print(f"⚠️ Endpoint /metrics no disponible: {e}")
        print(f"   ¿El server tiene prometheus-client instalado?")
        print(f"   pip install prometheus-client")

---
## 📋 Resumen y Acciones

### Comandos CLI Alternativos

Todos los módulos también funcionan como scripts CLI:

```bash
# Evaluación completa
python evaluation/evaluate_model.py --test-data ../FASE_2_DATASET/output/okla_test.jsonl --server-url http://localhost:8000

# Análisis de feedback
python feedback/feedback_system.py report --data-dir ./feedback/feedback_data

# Drift detection
python monitoring/drift_detector.py monitor --server-url http://localhost:8000 --baseline results/baseline.json

# A/B testing
python ab_testing/ab_testing.py analyze --experiment results/experiments/exp.json

# Retrain check
python retrain/retrain_pipeline.py check --versions-file results/model_versions.json
```

### 🔌 Workflow VS Code + Colab

| Tarea | Dónde ejecutar |
|-------|---------------|
| Evaluación rápida (< 50 samples) | **Local** |
| Evaluación completa (500+ samples) | **Colab** (más RAM/CPU) |
| Feedback analysis | **Local** |
| Drift detection | **Local** |
| Re-entrenamiento | **Colab con GPU** (FASE 3 notebook) |